In [4]:
import pandas as pd

In [5]:
df=pd.read_csv("/data2/ffp/datasets/trainingsets/newfull/traindataset_new_norm.csv")

In [6]:
df

,dom_vel,res_max,max_dew_temp,min_dew_temp,mean_dew_temp,max_temp,min_temp,mean_temp,rain_7_days,ndvi,evi,lst_day,lst_night,pop,slope,road_dens,dem,aspect
0,0.123914,1.044308,0.660740,0.625982,0.633226,0.633981,0.645109,0.634582,129.890517,0.851500,0.612083,0.60256,0.615913,0.004228,0.883978,0.119895,0.182014,0.537255
1,0.159099,1.045045,0.651779,0.593272,0.607723,0.635324,0.656973,0.652907,129.890517,0.761167,0.511083,0.63544,0.623304,0.004259,0.834254,0.276006,0.225076,0.184314
2,0.150598,1.076953,0.638552,0.598085,0.613687,0.625825,0.675640,0.652024,129.838973,0.631583,0.452167,0.64560,0.626087,0.010452,0.983425,0.013776,0.222903,0.258824
3,0.131888,1.055599,0.638184,0.583396,0.593459,0.623724,0.652090,0.638280,129.271988,0.801833,0.404667,0.60776,0.616000,0.003237,0.939227,0.097187,0.373894,0.313725
4,0.087409,1.026145,0.728094,0.624452,0.667115,0.740231,0.707829,0.736510,129.100175,0.703417,0.512083,0.60280,0.617217,0.010328,0.745856,0.037211,0.415752,0.572549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27191,0.071011,1.109352,0.709353,0.626898,0.676335,0.651412,0.671363,0.660968,135.680635,0.678000,0.442667,0.62232,0.585652,0.006118,0.994475,0.144366,0.098280,0.368627
27192,0.071011,1.109598,0.709353,0.626898,0.676335,0.651412,0.671363,0.660968,135.680635,0.746333,0.482500,0.62176,0.585739,0.005807,0.994475,0.091134,0.093271,0.494118
27193,0.071011,1.109843,0.709353,0.626898,0.676335,0.651412,0.671363,0.660968,135.680635,0.764500,0.480583,0.63968,0.583217,0.005674,0.977901,0.091134,0.080038,0.388235
27194,0.071011,1.109352,0.709353,0.626898,0.676335,0.651412,0.671363,0.660968,135.697816,0.602333,0.402083,0.62232,0.585652,0.006080,1.000000,0.144366,0.095567,0.000000


In [7]:
df.mean()

dom_vel            0.212675
res_max            1.106820
max_dew_temp       0.780956
min_dew_temp       0.726367
mean_dew_temp      0.742673
max_temp           0.749660
min_temp           0.795276
mean_temp          0.776222
rain_7_days      128.541792
ndvi               0.588334
evi                0.425765
lst_day            0.628470
lst_night          0.620076
pop                0.012250
slope              0.946384
road_dens          0.101522
dem                0.170100
aspect             0.491023
dtype: float64

In [10]:
df['rain_7_days'].min()

116.16261288841989